# Introduction

MedGemma is a multimodal specialized model from Google DeepMind. It is based on Gemma 3 model.

It has multiple utilizations, including:

* Medical image classification - can be used to classiffy images of various modalities: radiology, digital pathology, fundus and skin images.
* Medical image interpretation - can interpret a variety of medical images (various radiologic modalities as well as protocols) and answer natural language questions about medical images.
* Medical text comprehension and clinical reasoning - for most use cases, the larger 27B model will yields the best performance but also the 4B model can achieve good results.

This Notebook will demonstrate use of the MedGemma model to extract ICD10 codes from clinical notes.

## What are ICD10 codes?

ICD-10 stands for the International Classification of Diseases, 10th Revision. It is a global standard diagnostic tool for epidemiology, health management, and clinical purposes, developed and maintained by the World Health Organization (WHO).

ICD10 feature description:

* 🔢 **Format** - Codes are alphanumeric (e.g., E11.9, I10, K35.80)
* 📄 **Structure** - Each code represents a diagnosis, condition, symptom, or external cause
* 🌍 **Global Use** - Used by health systems in over 100 countries
* 💰 **Billing Support** - Required for insurance reimbursement and medical billing

Examples of ICD10 codes:

* **E11.9**	- Type 2 diabetes mellitus without complications
* **I10** -	Essential (primary) hypertension
* **K35.80** - Unspecified acute appendicitis
* **Z79.2**	- Long term (current) use of antibiotics
* **T36.0X1A** - Poisoning by penicillins, accidental (initial encounter)

# Preparation

Gemma 3 is supported from transformers 4.50.0. If your Notebook is preinstalled with an earlier version, you will need to upgrage transformers library:

`pip install -U transformers`

In [1]:
import os
import torch
import requests
from PIL import Image
import numpy as np
from transformers import pipeline
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login
from IPython.display import Markdown, display
import matplotlib.pyplot as plt

2025-05-31 20:35:57.982257: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748723758.471658      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748723758.590331      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Load the model

We will load the model from HuggingFace, since at the time when this Notebook was written, the model is not yet published on Kaggle.

The model is protected, therefore you will have to:
* Authenticate before accessing.
* Accept the terms and conditions to use it.  

You have few alternatives to access the model. On Kaggle, it is most convenient to use a access token, generated from HuggingFace and stored as a secret using `kaggle_secrets`.

## Authenticate to access the model

In [2]:
user_secrets = UserSecretsClient()
hugging_face_token = user_secrets.get_secret("HUGGINGFACE_TOKEN")

After you load the secret, you can login using huggingface_hub login function and with the authentication token as parameter.

In [3]:
login(hugging_face_token)

# Define a model pipeline  

Once authenticated, we can use transformers pipeline to initialize the model. In the pipeline initialization, we specify the mode **image-text-to-text**. For the application we need to implement, we will not necesarily need to include image as input but there is the option to pass the input as a scaned image.

In [4]:
pipe = pipeline(
    "image-text-to-text",
    model="google/medgemma-4b-it",
    torch_dtype=torch.bfloat16,
    device="cuda",
)

config.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

Device set to use cuda


# Load a clinical note

We initialize a string variable with the text of a clinical note.

In [5]:
clinical_note = """
Chief Complaint:
The patient presents with abdominal pain localized to the lower right quadrant, nausea, and low-grade fever for the past 24 hours.

History & Symptoms:
- Pain began near the umbilicus and migrated to the lower right abdomen
- Mild nausea, no vomiting
- Pain increases with movement or coughing
- Temperature: 38.1°C (100.6°F)

Physical Exam:
- Rebound tenderness in the right lower quadrant
- Positive Rovsing’s sign
- No palpable masses

Diagnosis:
- Acute uncomplicated appendicitis

Plan:
- Schedule for laparoscopic appendectomy
- Start IV fluids and antibiotics pre-op
"""

# Query the model

We prepare the model instructions.

First, let's prepare the system prompt. This will instruct the model to perform the extraction of ICD10 codes from a clinical note.

In [6]:
system_prompt = f"""
You are an expert clinical coder. From the following medical note, 
identify the most relevant ICD-10 codes:

Medical Note:
{clinical_note}

Instructions:
- Do not output one code more than once
- Try to include codes for all sections of the clinical note 
(complain, history and symptomps, diagnosis, plan)

Output format:
- **Code**: <code>, **Description**: <description>

Just output the ICD-10 code and description, in the format above mentioned.
"""

Let's initialize an empty image, since our pipeline will need also an image as input.

In [7]:
dummy_image = Image.fromarray(np.zeros((224, 224, 3), dtype=np.uint8))

Let's initialize the messages, including the system prompt and the user input.

In [8]:
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": system_prompt}]
    },
    {
        "role": "user",
        "content": [
            {"type": "text", "text": clinical_note},
             {"type": "image", "image": dummy_image}
            
        ]
    }
]

Let's query now the model.

In [9]:
output = pipe(text=messages, max_new_tokens=512)

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


Let's print the output.

In [10]:
display(Markdown(output[0]["generated_text"][-1]["content"]))

**Code**: K36.9, **Description**: Acute appendicitis, unspecified


# More examples

In [11]:
clinical_note = """
Chief Complaint:
- The patient reports fever, dry cough, and body aches for the past 2 days.

History & Symptoms:=
- Sudden onset of symptoms
- Fever peaked at 39°C (102.2°F)
- Chills and fatigue
- Headache and nasal congestion
- No sore throat or gastrointestinal symptoms
- No known COVID-19 exposure

Physical Exam:
- Temp: 38.7°C
- Mildly elevated heart rate
- Lungs: clear to auscultation
- No signs of bacterial infection

Diagnosis:
- Influenza, suspected viral etiology

Plan:
- Supportive care: hydration, rest, antipyretics
- Oseltamivir 75 mg BID for 5 days (started within 48 hours of symptom onset)
- Advise to return if shortness of breath or chest pain develops
"""

messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": system_prompt}]
    },
    {
        "role": "user",
        "content": [
            {"type": "text", "text": clinical_note},
             {"type": "image", "image": dummy_image}
            
        ]
    }
]

output = pipe(text=messages, max_new_tokens=512)
display(Markdown(output[0]["generated_text"][-1]["content"]))

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


**Code:** J11.9, **Description**: Influenza, unspecified


# Conclusions

We created a clinical note to ICD-10 codes extraction tool using MedGemma model.
We tested the solution with clinical notes provided as text.
The model is able to generate a set of ICD-10 codes.
Further validation of accuracy can be done using a LLM as a Judge pattern, for example by use of Gemini call.
